#### In this tutorial we will be characterising the frequency response of your TCLab unit. 

- Plot a Bode diagram of the FOPDT model you fit to your TCLab for the project
- Subject your TCLab sinusoidal variation of Q1 (Q₁ = Q̅ + Asin(ωt)) for at least 5 different values of ω chosen to be on both sides of the corner frequency of your FOPDT model.
- Find the amplitude ratio and phase angle for the responses to sinusoidal forcing and plot them as dots on your Bode diagram.
- Now subject your TCLab to a random set of pulses with long enough constant parts that the temperature has time to respond.
- Use the input Q1 and the output T1 together with numpy.fft.rfft to determine a numeric estimate of the frequency response. Plot this on the same graph as before.

Once you have characterised the open loop response of your unit, use the frequency response to find the largest gain that can be used for a PI controller with a time constant of 100 seconds (use the Bode stability criterion). Compare this to the answer you obtained in Tut 2. Was this process easier or harder than Tut 2? Which was more accurate?

#### Submit

- code in a notebook which you used to produce the response data
- code in a notebook which can be run to produce your resulting bode diagrams (note this should be separate).
- the data required to run the notebook above (your notebook should not require the TCLab to be connected).
- A two page report with graphs showing the results of your bode diagrams (system and system with controller) and including a brief discussion.

In [1]:
import tclab
from tclab import Plotter, clock, Historian
import datetime

import random

import numpy as np
import tbcontrol
from tbcontrol import blocksim

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
min_time = 50
limmit = 60*60*2
Qlower = 0
Qhigh = 100
start_delay = 300

In [4]:
time_span = [start_delay]
total_time = start_delay
while total_time<= limmit:
    time_random = random.randint(min_time, min_time*6)
    total_time += time_random
    time_span.append(time_random)

In [5]:
#time_span = array of run lengths in seconds

In [ ]:
run_length_spans = []
conned = True

with tclab.Experiment(connected = conned, synced = conned, plot = False) as ex:
    
    sources = [('T1', lambda: ex.lab.T1),
                ('Q1', lambda: ex.lab.U1)]
    
    h = tclab.Historian(sources)
    p = tclab.Plotter(h, total_time)
    
    t_passed = 0
    
    for index,run_length in enumerate(time_span):
        
        ex.time = run_length
        
        for t in ex.clock():
            
            Q = Qhigh if index%2 ==0 else Qlower
            
            ex.lab.Q1(Q)
            
#             print (run_length, Q, t)
            run_length_spans.append(run_length)
            p.update(t + t_passed)# + index*run_length)
            
        t_passed += run_length

TCLab version 0.4.9


In [ ]:
h.get_sessions()

In [ ]:
h.load_session(1)

In [ ]:
data = {"Time" : h.t, "T1" : h.logdict['T1'], "Q1" : h.logdict['Q1']}

In [ ]:
for entry in data:
    print (len(data[entry]))

In [ ]:
data_frame = pd.DataFrame(data = data)
data_frame.to_csv("Pulses.csv")

In [ ]:
data_frame.plot(y = "T1")
data_frame.plot(y = "Q1")